In [1]:
import pandas as pd
import numpy as np
from sklearnex import patch_sklearn
patch_sklearn()
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
import torch
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import json
import time
import torch.optim as optim
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
c:\Users\bruno\.conda\envs\my_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# Define a list of filenames to load
filenames = ["../data/labeled_data/generic_test_0.json"]

# Load all JSON data and concatenate into one DataFrame
dfs = []
for filename in filenames:
    with open(filename) as f:
        data = json.load(f)
    df_train = pd.DataFrame(data["train"])
    df_validation = pd.DataFrame(data["valid"])
    df_test = pd.DataFrame(data["test"])

In [13]:
df_train

,id,campaign_name,text,annotations
0,891103871484870657,VENEZUELA_201901_2,How life may find a way on Saturn's moon https...,[Science/Technology]
1,898650367067664384,VENEZUELA_201901_2,RightWing Millennial Retweet RT RT_America to ...,[Others]
2,1012051694136512512,IRA_202012,The tobacco industry then peddles their produc...,"[Health, Justice/Crime, Macroeconomics/Economi..."
3,1296389394883063810,UGANDA_0621,"RT @brianmixologist: Today, i want to salute a...",[Others]
4,1286189347973279746,UGANDA_0621,RT @ArthurMirama: You can only under estimate ...,[Government/Public]
...,...,...,...,...
3195,1090997040258187266,REA_0621,SABC will not carry out a staff reduction\nhtt...,[Labor/Employment]
3196,818131935117443074,VENEZUELA_201901_2,Wikileaks To Hold Major Press Conf To Address ...,[Others]
3197,827160940944646145,VENEZUELA_201901_2,Death of veteran DRC opposition leader jeopard...,[Government/Public]
3198,896689467863511040,VENEZUELA_201901_2,#lost RT realDonaldTrump: Condolences to the f...,"[Government/Public, Justice/Crime]"


In [14]:
df_validation

,id,campaign_name,text,annotations
0,962251540617670661,GRU_202012,RT @Kasman62: After his injury for fifth time ...,[War/Terror]
1,898277432922263552,VENEZUELA_201901_2,RT BGEEZ: dncchuckschumernancypelosimsnbccnn h...,[Others]
2,966214616601841664,GRU_202012,RT @LinaArabii: Russian presence in the Black ...,[Media/Journalism]
3,1303856635668987905,UGANDA_0621,RT @brianmixologist: Am disappointed in the an...,[Election Campaign]
4,462250037972840449,IRA_202012,RT @CMCL1979: Why do those those ghastly nativ...,[Others]
...,...,...,...,...
795,847754640972070912,VENEZUELA_201901_2,Stocks: 5 things to know before the bell https...,[Others]
796,1038043813267296256,GRU_202012,"September 4, 2018 #Syrian air defense units re...",[War/Terror]
797,1030464264748728320,UGANDA_0621,RT @xJ57jjSHWvX9mAMmhv7fVaVzxe13bBfCZuGZaBNucL...,[Others]
798,1020843977669586945,UGANDA_0621,RT @HowweEnt: Dj Shiru To Thrill His Fans http...,[Others]


In [15]:
df_test

,id,campaign_name,text,annotations
0,1144169368227635200,REA_0621,The Automobile Association said it is expectin...,[Macroeconomics/Economic Regulation]
1,1187637103850610688,REA_0621,"A severe flooding, triggered by heavy rains an...",[Environment]
2,1062269743749566464,GRU_202012,Militants of the Hayat Tahrir ash-Sham group p...,[War/Terror]
3,1100425181946880000,REA_0621,PIC Amendment Bill for transparency adopted by...,[Government/Public]
4,1129575563029561344,UGANDA_0621,RT @CQvMyyB0YfvwrUrsaZ6KI7yqaJfSUDTrAI0joQhgMA...,[Government/Public]
...,...,...,...,...
995,1166999406190780417,REA_0621,Rwanda and Zimbabwe entering deal: A Memorandu...,[Macroeconomics/Economic Regulation]
996,887818127295361024,VENEZUELA_201901_2,Trump Supporters React to Unhinged Rosie THREA...,[Government/Public]
997,495089817923756032,IRA_202012,The American Aggression Enablement Act and the...,[Others]
998,1154775632871477248,REA_0621,Body of a missing 4-year-old girl has been fou...,[Justice/Crime]


In [4]:
tasks=['generic', 'GRU_202012', 'IRA_202012', 'REA_0621', 'UGANDA_0621', 'VENEZUELA_201901']

# Train and evaluate your model using k-fold cross-validation
k = 5
results = []
for i in range(k):
    # Load the data for this fold
    filename = f"../data/labeled_data/generic_test_{i}.json"
    with open(filename) as f:
        data = json.load(f)
    train_df = pd.DataFrame(data["train"])
    val_df = pd.DataFrame(data["validation"])
    test_df = pd.DataFrame(data["test"])

    # Train and evaluate your model on this fold
    # ...

    # Store the results for this fold
    # ...
    results.append(fold_result)

# Average the results across all k folds
avg_result = np.mean(results)

FileNotFoundError: [Errno 2] No such file or directory: '../data/labeled_data/generic_fold_0.json'

In [16]:
class TweetDataset(Dataset):
    def __init__(self, df, y):
        self.input_ids = df['input_ids'].tolist()
        self.attention_mask = df['attention_mask']
        self.labels = y

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_mask[idx], self.labels[idx]
    
class TweetClassifier(nn.Module):
    def __init__(self, num_labels):
        super(TweetClassifier, self).__init__()
        self.bertweet = AutoModel.from_pretrained("vinai/bertweet-large")
        self.linear = nn.Linear(self.bertweet.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        output = self.bertweet(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = output.pooler_output
        logits = self.linear(pooled_output)
        return logits

In [29]:
len(y_train)

3200

In [36]:
# Convert annotations column to a list of labels
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(df_train['annotations'])
y_validation = mlb.fit_transform(df_validation['annotations'])
y_test = mlb.fit_transform(df_test['annotations'])

print(y)

# Load BERTweet model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large", normalization=True)
model = AutoModel.from_pretrained("vinai/bertweet-large")

# Preprocess the text column by encoding the tweets and adding special tokens
def preprocess_text(text):
    return tokenizer.encode_plus(text, max_length=128, add_special_tokens=True, truncation=True, padding='max_length', return_attention_mask=True, return_tensors='pt')

encoded_tweets = df_train['text'].apply(preprocess_text)
input_ids = encoded_tweets.apply(lambda x: x['input_ids']).to_list()
attention_mask = encoded_tweets.apply(lambda x: x['attention_mask']).to_list()
df_encoded_tweets_train = pd.DataFrame({'input_ids': input_ids, 'attention_mask': attention_mask})

encoded_tweets = df_validation['text'].apply(preprocess_text)
input_ids = encoded_tweets.apply(lambda x: x['input_ids']).to_list()
attention_mask = encoded_tweets.apply(lambda x: x['attention_mask']).to_list()
df_encoded_tweets_validation = pd.DataFrame({'input_ids': input_ids, 'attention_mask': attention_mask})

encoded_tweets = df_validation['text'].apply(preprocess_text)
input_ids = encoded_tweets.apply(lambda x: x['input_ids']).to_list()
attention_mask = encoded_tweets.apply(lambda x: x['attention_mask']).to_list()
df_encoded_tweets_test = pd.DataFrame({'input_ids': input_ids, 'attention_mask': attention_mask})

train_dataset = TweetDataset(df_encoded_tweets_train.reset_index(), torch.tensor(y_train))
validation_dataset = TweetDataset(df_encoded_tweets_validation.reset_index(), torch.tensor(y_validation))
test_dataset = TweetDataset(df_encoded_tweets_test.reset_index(), torch.tensor(y_test))

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]


Some weights of the model checkpoint at vinai/bertweet-large were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use 

In [32]:
model_name = "bertweet-large-base"

In [33]:
len(train_loader) # len(daatset) / batch_size

800

In [9]:
# Train the classifier model using binary cross-entropy loss and the AdamW optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TweetClassifier(num_labels=len(mlb.classes_)).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=0.001)

# Initialize dataframes for logging the losses and metrics
metrics = pd.DataFrame(columns=['epoch', 'train_loss', 'test_loss', 'precision_macro', 'precision_micro', 'recall_macro', 'recall_micro', 'f1_macro', 'f1_micro', 'accuracy', 'time'])

epochs = 200
scaler = torch.cuda.amp.GradScaler()

best_loss = float("inf")
best_model = None

#TODO: implement early stopping with patience = 1200 steps = 1.5 epochs
patience = 1200
early_stopping_counter = 0

eval_steps = 400

for epoch in range(epochs):
    global_step = 0
    start_time = time.time()
    model.train()
    train_loss = 0
    for batch in train_loader:
        input_ids = batch[0].to(device).squeeze(1)
        attention_mask = batch[1].to(device).squeeze(1)
        labels = batch[2].to(device)
        optimizer.zero_grad()
        
        # Casts operations to mixed precision
        with torch.cuda.amp.autocast():
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels.float())
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item()
        
        global_step += 1
        
        # Evaluate the model every eval_steps
        if global_step % eval_steps == 0:
            model.eval()
            test_loss = 0
            y_true = []
            y_pred = []

            with torch.no_grad():
                for batch in test_loader:
                    input_ids = batch[0].to(device).squeeze(1)
                    attention_mask = batch[1].to(device).squeeze(1)
                    labels = batch[2].to(device)
                    outputs = model(input_ids, attention_mask)
                    loss = criterion(outputs, labels.float())
                    test_loss += loss.item()

                    batch_pred = torch.sigmoid(outputs).cpu().detach().numpy()
                    labels = labels.cpu().detach().numpy()
                    y_true.append(labels)
                    y_pred.append(batch_pred)

            test_loss /= len(test_loader)
            
            y_true = np.vstack(y_true)
            y_pred = np.vstack(y_pred)
            precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_true, y_pred > 0.5, average='macro')
            precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(y_true, y_pred > 0.5, average='micro')
            accuracy = accuracy_score(y_true, y_pred > 0.5)
            
            elapsed_time = time.time() - start_time
            minutes, seconds = divmod(elapsed_time, 60)
            time_str = f"{int(minutes):02d}:{int(seconds):02d}"
            
            train_loss /= global_step

            new_row = pd.DataFrame({'epoch': [epoch+1], 'train_loss': [train_loss], 'test_loss': [test_loss], 'precision_macro': [precision_macro], 'precision_micro': [precision_micro], 'recall_macro': [recall_macro], 'recall_micro': [recall_micro], 'f1_macro': [f1_macro], 'f1_micro': [f1_micro], 'accuracy': [accuracy], 'time': [time_str]})
            metrics = pd.concat([metrics, new_row], ignore_index=True)  
            print(f'Epoch {epoch+1}/{epochs}, Step {global_step}, Train Loss: {train_loss}, Test Loss: {test_loss}, Precision Macro: {precision_macro}, Precision Micro: {precision_micro}, Recall Macro: {recall_macro}, Recall Micro: {recall_micro}, F1 Macro: {f1_macro}, F1 Micro: {f1_micro}, Accuracy: {accuracy}, Time: {time_str}')

            # Save the best model based on the evaluation loss
            #TODO: change to f1-micro
            if test_loss < best_loss:
                print("Best Current Model Found, Saving Model...")
                best_loss = test_loss
                best_model = model.state_dict()
                early_stopping_counter = 0

            model.train()
            
# Load the best model
model.load_state_dict(torch.load("../models/{model_name}.pt"))


Some weights of the model checkpoint at vinai/bertweet-large were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use 

Epoch 1/200, Step 2, Train Loss: 0.6515279710292816, Test Loss: 0.6006966147571802, Precision Macro: 0.06019807314462351, Precision Micro: 0.15666666666666668, Recall Macro: 0.2516214888973418, Recall Micro: 0.4297142857142857, F1 Macro: 0.08104107512248815, F1 Micro: 0.22961832061068702, Accuracy: 0.0109375, Time: 00:04
Best Current Model Found, Saving Model...


c:\Users\bruno\.conda\envs\my_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bruno\AppData\Local\Temp\ipykernel_31328\3797979436.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics = metrics.append({'epoch': epoch+1, 'train_loss': train_loss, 'test_loss': test_loss, 'precision_macro': precision_macro, 'precision_micro': precision_micro, 'recall_macro': recall_macro, 'recall_micro': recall_micro, 'f1_macro': f1_macro, 'f1_micro': f1_micro, 'accuracy': accuracy, 'time': time_str}, ignore_index=True)


Epoch 1/200, Step 4, Train Loss: 0.46678126603364944, Test Loss: 0.5553391557186842, Precision Macro: 0.056026310045576885, Precision Micro: 0.18873239436619718, Recall Macro: 0.09681316518661137, Recall Micro: 0.2297142857142857, F1 Macro: 0.04881403535759021, F1 Micro: 0.2072164948453608, Accuracy: 0.165625, Time: 00:10
Best Current Model Found, Saving Model...


c:\Users\bruno\.conda\envs\my_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bruno\AppData\Local\Temp\ipykernel_31328\3797979436.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics = metrics.append({'epoch': epoch+1, 'train_loss': train_loss, 'test_loss': test_loss, 'precision_macro': precision_macro, 'precision_micro': precision_micro, 'recall_macro': recall_macro, 'recall_micro': recall_micro, 'f1_macro': f1_macro, 'f1_micro': f1_micro, 'accuracy': accuracy, 'time': time_str}, ignore_index=True)


...

c:\Users\bruno\.conda\envs\my_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bruno\AppData\Local\Temp\ipykernel_31328\3797979436.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics = metrics.append({'epoch': epoch+1, 'train_loss': train_loss, 'test_loss': test_loss, 'precision_macro': precision_macro, 'precision_micro': precision_micro, 'recall_macro': recall_macro, 'recall_micro': recall_micro, 'f1_macro': f1_macro, 'f1_micro': f1_micro, 'accuracy': accuracy, 'time': time_str}, ignore_index=True)


Epoch 1/200, Step 24, Train Loss: 0.03485536977041193, Test Loss: 0.360365048982203, Precision Macro: 0.05933806146572104, Precision Micro: 0.27927927927927926, Recall Macro: 0.010933811356979831, Recall Micro: 0.03542857142857143, F1 Macro: 0.016724772774536066, F1 Micro: 0.06288032454361055, Accuracy: 0.0125, Time: 01:01
Best Current Model Found, Saving Model...


c:\Users\bruno\.conda\envs\my_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bruno\AppData\Local\Temp\ipykernel_31328\3797979436.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics = metrics.append({'epoch': epoch+1, 'train_loss': train_loss, 'test_loss': test_loss, 'precision_macro': precision_macro, 'precision_micro': precision_micro, 'recall_macro': recall_macro, 'recall_micro': recall_micro, 'f1_macro': f1_macro, 'f1_micro': f1_micro, 'accuracy': accuracy, 'time': time_str}, ignore_index=True)


Epoch 1/200, Step 26, Train Loss: 0.025951006246067172, Test Loss: 0.35531490873545407, Precision Macro: 0.05933806146572104, Precision Micro: 0.24752475247524752, Recall Macro: 0.008594630070430124, Recall Micro: 0.02857142857142857, F1 Macro: 0.012569496667373685, F1 Micro: 0.051229508196721306, Accuracy: 0.0125, Time: 01:06
Best Current Model Found, Saving Model...


KeyboardInterrupt: 

In [29]:
metrics

,epoch,train_loss,test_loss,precision_macro,precision_micro,recall_macro,recall_micro,f1_macro,f1_micro,accuracy,time
0,1,0.216469,0.152939,0.487925,0.840989,0.266169,0.544000,0.321336,0.660652,0.475000,02:12
1,2,0.127269,0.121936,0.810334,0.849138,0.467086,0.675429,0.550905,0.752387,0.584375,02:10
2,3,0.089811,0.109567,0.768305,0.860497,0.578644,0.712000,0.637684,0.779237,0.615625,02:09
3,4,0.065169,0.109837,0.779510,0.828283,0.620807,0.749714,0.674415,0.787043,0.631250,02:12
4,5,0.048520,0.106363,0.766207,0.824561,0.635792,0.752000,0.675435,0.786611,0.631250,02:09
5,6,0.034025,0.106338,0.759219,0.811138,0.659796,0.765714,0.694594,0.787772,0.631250,02:07
6,7,0.026678,0.114554,0.769001,0.801190,0.640851,0.769143,0.674094,0.784840,0.620313,02:07
7,8,0.020222,0.125085,0.757733,0.811414,0.633249,0.747429,0.667265,0.778108,0.625000,02:09
8,9,0.019148,0.122573,0.738763,0.799770,0.672417,0.794286,0.692426,0.797018,0.639062,02:09
9,10,0.019440,0.122291,0.750067,0.803738,0.676510,0.786286,0.699300,0.794916,0.639062,02:08


In [32]:
folder_path = "../models"
model_name = "bertweet_large_mlb.pt"

import os
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
torch.save(model.state_dict(), os.path.join(folder_path, model_name))